In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import pandas as pd
import tensorflow as tf
import numpy as np

import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import session
from sqlalchemy import create_engine, func
import matplotlib.pyplot as plt

In [2]:
#connecting to SQL DB
host_address = 'bootcamp-final-project.c8u2worjd1ui.us-east-1.rds.amazonaws.com'
port = '5432'
username = 'peter_jennifer'
password = 'Puhj6k2%pbW'
db = 'us_gun_violence'


In [3]:
# A long string that contains the necessary Postgres login information
postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'.format(
                username=username,
                password=password,
                ipaddress=host_address,
                port=port,
                dbname=db))
# Create the connection
engine = create_engine(postgres_str)

In [4]:
#make DBs
suspects_df = pd.read_sql_query('''SELECT * FROM suspects;''', engine)
incidents_df = pd.read_sql_query('''SELECT * FROM incidents;''', engine)
guns_df = pd.read_sql_query('''SELECT * FROM guns;''', engine)

guns_df_dup = guns_df.drop_duplicates(subset='incident_id')
guns_df_dup

,gun_index,incident_id,n_guns_involved,gun_stolen,gun_type
0,0,478855,2,Unknown,Unknown
2,2,478959,2,Unknown,Handgun
4,4,479363,2,Unknown,22 LR
6,6,491674,1,Unknown,Unknown
7,7,479413,1,Unknown,Unknown
...,...,...,...,...,...
192446,192446,1083142,1,Unknown,Unknown
192447,192447,1083139,1,Unknown,Unknown
192448,192448,1083151,1,Unknown,Unknown
192449,192449,1082514,1,Unknown,Unknown


In [5]:
guns_df_dup.gun_stolen.value_counts()
gun_ownership = guns_df_dup.loc[(guns_df_dup['gun_stolen'] == 'Stolen') | (guns_df_dup['gun_stolen'] == 'Not-stolen')] 

gun_owners_df = gun_ownership.merge(incidents_df, on= 'incident_id', how='left')

gun_owners_df

,gun_index,incident_id,n_guns_involved,gun_stolen,gun_type,date,state,latitude,longitude,n_killed,n_injured,incident_characteristics,notes,congressional_district,state_house_district,state_senate_district
0,26,484333,2,Not-stolen,40 SW,2013-04-21,Washington,47.3030,-122.3100,5,0,"Shot - Dead (murder, accidental, suicide)||Hom...",None,9.0,30.0,30.0
1,51,489581,2,Not-stolen,223 Rem [AR-15],2013-06-07,California,34.0260,-118.4610,6,1,"Shot - Wounded/Injured||Shot - Dead (murder, a...","ms; 1 inj, 6 killed, including shooter. Incide...",33.0,50.0,26.0
2,99,480691,2,Stolen,Handgun,2013-09-24,West Virginia,38.4136,-82.4609,0,6,Shot - Wounded/Injured||Mass Shooting (4+ vict...,None,3.0,17.0,5.0
3,164,93918,2,Stolen,22 LR,2014-01-07,Virginia,36.8802,-80.0085,0,2,Shot - Wounded/Injured||Home Invasion||Officer...,.22-caliber revolver and a Mauser hunting rifle,5.0,9.0,19.0
4,167,94452,1,Not-stolen,Handgun,2014-01-07,New Hampshire,43.3062,-70.9721,0,1,Shot - Wounded/Injured||Accidental Shooting||A...,accidentally discharged handgun while trying t...,1.0,NaN,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8576,192240,1083028,5,Stolen,Unknown,2018-03-30,North Carolina,36.4126,-77.6172,0,0,Non-Shooting Incident||ATF/LE Confiscation/Rai...,None,1.0,27.0,4.0
8577,192295,1082574,1,Stolen,9mm,2018-03-30,California,37.8127,-121.2130,0,0,Non-Shooting Incident||Gang involvement||ATF/L...,None,10.0,12.0,5.0
8578,192302,1083233,8,Stolen,357 Mag,2018-03-30,Colorado,NaN,NaN,0,0,Non-Shooting Incident||Gun(s) stolen from owner,Guns stolen from shed.,NaN,NaN,NaN
8579,192316,1081863,1,Stolen,Handgun,2018-03-31,Ohio,NaN,NaN,0,0,Non-Shooting Incident||Gun(s) stolen from owner,Gun stolen from residence.,NaN,NaN,NaN


In [6]:
suspects_age_df = suspects_df.dropna(subset=['participant_age'], axis=0)

In [7]:
suspects_age_df

,suspect_index,incident_id,participant_gender,participant_age,participant_age_group,participant_status
1,1,478855,Male,25.0,Adult 18+,"Injured, Unharmed, Arrested"
2,2,478855,Male,31.0,Adult 18+,"Unharmed, Arrested"
3,3,478925,Male,33.0,Adult 18+,Killed
4,4,478959,Female,47.0,Adult 18+,Killed
7,7,479363,Male,15.0,Teen 12-17,"Unharmed, Arrested"
...,...,...,...,...,...,...
193243,193243,1082234,Female,35.0,Adult 18+,Arrested
193245,193245,1082061,Male,48.0,Adult 18+,"Unharmed, Arrested"
193246,193246,1083142,Female,25.0,Adult 18+,"Unharmed, Arrested"
193247,193247,1083139,Male,21.0,Adult 18+,"Unharmed, Arrested"


In [8]:
outcomes = suspects_age_df['participant_status'].value_counts()
replace_type = list(outcomes[outcomes < 2000].index)

for item in replace_type:
    suspects_age_df.participant_status = suspects_age_df.participant_status.replace(item, "Other")

suspects_age_df['participant_status'].value_counts()

suspects_age_df = suspects_age_df.drop(columns='participant_age_group')

/Users/jennifertian/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/generic.py:5208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [9]:
df_cat = suspects_age_df.dtypes[suspects_age_df.dtypes == 'object'].index.tolist()

In [13]:
#Encode values

encode_df = pd.get_dummies(suspects_age_df, columns= df_cat)
encode_df.head()

,suspect_index,incident_id,participant_age,participant_gender_Female,participant_gender_Male,participant_status_Arrested,participant_status_Injured,"participant_status_Injured, Arrested",participant_status_Killed,participant_status_Other,participant_status_Unharmed,"participant_status_Unharmed, Arrested"
1,1,478855,25.0,0,1,0,0,0,0,1,0,0
2,2,478855,31.0,0,1,0,0,0,0,0,0,1
3,3,478925,33.0,0,1,0,0,0,1,0,0,0
4,4,478959,47.0,1,0,0,0,0,1,0,0,0
7,7,479363,15.0,0,1,0,0,0,0,0,0,1
